<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-18 11:24:51
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.37 C
-------------------
Today PnL: 2.62 L (1.95%)
Current PnL: -18.18 L (-12.36%)
CY Booked + Current PnL: -9.91 L (-6.74%)
-------------------
Total profit:  3.66 L
Total loss:  -21.84 L
-------------------
Total Booked + Current PnL: 17.07 L (14.22%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.05%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 77.65 L (56.81%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-17.42,H-LC,6.53,113648.0,9336.0,7319.0,2.16,8.95,6.44,15.96,11.0,1.28,0.85,27.36,X5K,ATH,METALS
68,SBIN,760.30,863.00,-11.34,M-LC,3.87,219950.0,18470.0,8754.0,0.42,9.17,3.98,13.51,61.0,2.11,1.64,22.92,XY25,NTT,BANKS
51,LALPATHLAB,2838.53,3545.00,-22.35,H-SC,10.90,231866.0,38846.0,9205.0,1.85,20.13,3.97,24.89,134.0,4.22,1.73,48.17,X40N,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-13.42,H-LC,6.27,173063.0,16886.0,9207.0,0.58,10.81,5.32,16.71,19.0,1.83,1.29,29.97,X40N,BTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,10.54,H-MC,13.91,157737.0,31537.0,14638.0,-1.04,24.99,9.28,36.59,111.0,2.15,1.18,40.64,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJOOENG,102.50,143.33,-32.41,H-SC,6.03,102500.0,0.0,40826.0,-0.06,0.00,39.83,39.83,116.0,0.00,0.76,1.49,AR,ATH,MISC
58,PGHH,13388.00,18062.58,-30.28,H-MC,6.23,201945.0,1125.0,68984.0,1.47,0.56,34.16,34.92,82.0,0.02,1.50,5.26,X40,ATH,FMCG
56,NATIONALUM,189.63,247.44,-49.46,H-MC,1.99,100059.0,-445.0,31088.0,0.65,-0.44,31.07,30.49,80.0,-0.01,0.75,31.82,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-19.78,H-LC,2.79,213597.0,9097.0,32317.0,-0.31,4.45,15.13,20.25,10.0,0.28,1.59,13.57,X40N,BTT,PHARMA
57,NESTLEIND,1134.30,1377.0,-11.66,H-LC,3.44,271861.0,6435.0,50349.0,6.65,2.42,18.52,21.40,8.0,0.13,2.03,9.93,XY25,NTT,FMCG
53,LTIM,5564.16,7230.2,-7.51,H-LC,3.73,183204.0,-17106.0,77092.0,-0.37,-8.54,42.08,29.94,17.0,-0.22,1.37,26.57,X200,ATH,IT
5,ASIANPAINT,2961.56,4250.0,-9.79,H-LC,3.73,176011.0,-25375.0,112981.0,2.36,-12.60,64.19,43.51,27.0,-0.22,1.31,22.52,X40,BTT,PAINTS
37,HINDUNILVR,2413.81,2723.0,-8.05,H-LC,5.96,289016.0,18669.0,15954.0,4.03,6.91,5.52,12.81,28.0,1.17,2.15,20.87,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELAXO,902.64,1176.00,-32.42,H-SC,10.59,62726.0,-52812.0,87804.0,12.24,-45.71,139.98,30.28,136.0,-0.60,0.47,22.54,X40N,NTT,FOOTWEAR
15,BLUESTARCO,1646.70,2326.38,11.47,H-MC,6.56,192370.0,27700.0,40263.0,7.76,16.82,20.93,41.28,91.0,0.69,1.43,26.15,X40N,ATH,AC
90,VOLTAS,1278.28,1918.49,-2.34,H-MC,7.62,204885.0,13143.0,82896.0,7.15,6.85,40.46,50.08,101.0,0.16,1.53,14.12,XY25,ATH,AC
57,NESTLEIND,1134.30,1377.00,-11.66,H-LC,3.44,271861.0,6435.0,50349.0,6.65,2.42,18.52,21.40,8.0,0.13,2.03,9.93,XY25,NTT,FMCG
13,BATAINDIA,1550.24,2096.00,-37.87,M-SC,7.48,92819.0,-35851.0,81152.0,6.54,-27.86,87.43,35.20,209.0,-0.44,0.69,6.54,X40,NTT,FOOTWEAR


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,SONACOMS,578.05,1006.00,-37.35,M-SC,13.61,77989.0,-23170.0,98063.0,-1.37,-22.90,125.74,74.03,203.0,-0.24,0.58,9.97,AR,BTT,AUTO
89,VBL,492.64,671.64,-12.21,H-LC,10.07,266444.0,1404.0,94907.0,-1.32,0.53,35.62,36.33,5.0,0.01,1.99,13.90,X40N,ATH,FMCG
55,MEDANTA,1087.93,1486.00,10.54,H-MC,13.91,157737.0,31537.0,14638.0,-1.04,24.99,9.28,36.59,111.0,2.15,1.18,40.64,XY24,NTT,HEALTHCARE
3,ALKYLAMINE,2347.98,4546.37,-4.17,H-SC,15.58,89152.0,-11811.0,106341.0,-0.94,-11.70,119.28,93.63,149.0,-0.11,0.66,32.79,SR,ATH,CHEMICALS
45,INFY,1461.46,2275.00,-21.39,H-LC,11.26,257688.0,-3913.0,149536.0,-0.56,-1.50,58.03,55.67,4.0,-0.03,1.92,4.52,X40,BTT,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,95.94,M-SC,10.13,84403.0,-16374.0,16467.0,1.99,-16.25,19.51,0.09,243.0,-0.99,0.63,10.89,OX40N,NTT,DURABLES
20,CERA,7989.07,9475.0,-17.90,H-SC,3.14,117225.0,-26578.0,53326.0,1.79,-18.48,45.49,18.60,148.0,-0.50,0.87,29.36,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,114.84,M-SC,2.56,140963.0,-33596.0,33634.0,2.67,-19.25,23.86,0.02,221.0,-1.00,1.05,22.24,OX40N,NTT,PAINTS
84,UJJIVANSFB,52.77,60.0,80.86,H-SC,20.12,114291.0,-28188.0,47705.0,1.58,-19.78,41.74,13.70,163.0,-0.59,0.85,34.89,OX40N,NTT,BANKS
49,KANSAINER,299.63,340.0,-69.07,H-SC,3.05,213210.0,-56457.0,92789.0,0.62,-20.94,43.52,13.47,142.0,-0.61,1.59,7.84,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.69,H-LC,16.09,235615.0,-60319.0,154234.0,-0.05,-20.38,65.46,31.73,2.0,-0.39,1.76,0.59,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.39,H-LC,11.26,257688.0,-3913.0,149536.0,-0.56,-1.50,58.03,55.67,4.0,-0.03,1.92,4.52,X40,BTT,IT
89,VBL,492.64,671.64,-12.21,H-LC,10.07,266444.0,1404.0,94907.0,-1.32,0.53,35.62,36.33,5.0,0.01,1.99,13.90,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.30,H-LC,10.82,196053.0,-20466.0,113377.0,-0.06,-9.45,57.83,42.91,7.0,-0.18,1.46,2.67,AR,NTT,ELECTRICAL
57,NESTLEIND,1134.30,1377.00,-11.66,H-LC,3.44,271861.0,6435.0,50349.0,6.65,2.42,18.52,21.40,8.0,0.13,2.03,9.93,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.0,-22.35,H-SC,10.90,231866.0,38846.0,9205.0,1.85,20.13,3.97,24.89,134.0,4.22,1.73,48.17,X40N,NTT,HEALTHCARE
55,MEDANTA,1087.93,1486.0,10.54,H-MC,13.91,157737.0,31537.0,14638.0,-1.04,24.99,9.28,36.59,111.0,2.15,1.18,40.64,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-10.37,H-LC,6.65,223778.0,39297.0,31060.0,4.79,21.30,13.88,38.14,35.0,1.27,1.67,34.16,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,1646.70,2326.38,11.47,H-MC,6.56,192370.0,27700.0,40263.0,7.76,16.82,20.93,41.28,91.0,0.69,1.43,26.15,X40N,ATH,AC
55,MEDANTA,1087.93,1486.00,10.54,H-MC,13.91,157737.0,31537.0,14638.0,-1.04,24.99,9.28,36.59,111.0,2.15,1.18,40.64,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,10.17,H-SC,11.68,131292.0,5995.0,53003.0,2.43,4.78,40.37,47.09,141.0,0.11,0.98,64.32,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-2.34,H-MC,7.62,204885.0,13143.0,82896.0,7.15,6.85,40.46,50.08,101.0,0.16,1.53,14.12,XY25,ATH,AC
19,CAMS,3643.00,5211.76,-4.72,H-SC,3.87,108321.0,6317.0,37609.0,1.24,6.19,34.72,43.06,123.0,0.17,0.81,24.37,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-5.37,H-MC,8.61,175448.0,24535.0,32774.0,6.47,16.26,18.68,37.97,92.0,0.75,1.31,45.68,AR,ATH,AUTO
50,KPIGREEN,497.21,731.35,10.17,H-SC,11.68,131292.0,5995.0,53003.0,2.43,4.78,40.37,47.09,141.0,0.11,0.98,64.32,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-51.84,H-SC,11.79,134588.0,11252.0,122515.0,-0.25,9.12,91.03,108.45,119.0,0.09,1.00,33.52,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-42.31,M-SC,4.52,97492.0,5994.0,72758.0,4.77,6.55,74.63,86.07,219.0,0.08,0.73,41.03,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-12.89,M-LC,7.60,153134.0,2189.0,107270.0,0.07,1.45,70.05,72.51,54.0,0.02,1.14,7.56,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.43,74074.0,-39475.0,79526.0,1.63,-34.76,107.36,35.27,268.0,-0.50,0.55,81.30,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.17,H-SC,15.58,89152.0,-11811.0,106341.0,-0.94,-11.70,119.28,93.63,149.0,-0.11,0.66,32.79,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-18.25,H-SC,13.26,92985.0,-4995.0,26863.0,3.14,-5.10,28.89,22.32,152.0,-0.19,0.69,34.86,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-325.20,H-SC,16.42,95680.0,-37606.0,122202.0,-0.15,-28.21,127.72,63.47,143.0,-0.31,0.71,60.38,XR,NTT,CHEMICALS
9,BAJAJFINSV,1994.80,2758.00,-8.50,H-LC,7.23,98304.0,559.0,36835.0,4.21,0.57,37.47,38.26,38.0,0.02,0.73,29.40,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-5.37,H-MC,8.61,175448.0,24535.0,32774.0,6.47,16.26,18.68,37.97,92.0,0.75,1.31,45.68,AR,ATH,AUTO
51,LALPATHLAB,2838.53,3545.00,-22.35,H-SC,10.90,231866.0,38846.0,9205.0,1.85,20.13,3.97,24.89,134.0,4.22,1.73,48.17,X40N,NTT,HEALTHCARE
28,DMART,3764.92,5201.00,-10.37,H-LC,6.65,223778.0,39297.0,31060.0,4.79,21.30,13.88,38.14,35.0,1.27,1.67,34.16,X40N,NTT,FMCG
5,ASIANPAINT,2961.56,4250.00,-9.79,H-LC,3.73,176011.0,-25375.0,112981.0,2.36,-12.60,64.19,43.51,27.0,-0.22,1.31,22.52,X40,BTT,PAINTS
4,APOLLOHOSP,7098.95,8285.00,-13.42,H-LC,6.27,173063.0,16886.0,9207.0,0.58,10.81,5.32,16.71,19.0,1.83,1.29,29.97,X40N,BTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.63
1,25,41.40
2,50,71.26


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.85
LC    34.78
MC    26.36
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.45
X40N     15.53
X40      13.27
XY25     11.72
XR        8.93
AR        8.27
OX40N     7.39
X5K       2.24
MH        1.73
X200      1.37
SR        1.09
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.55
H-SC    24.17
H-MC    23.45
M-SC    13.22
M-LC     7.17
M-MC     2.57
L-SC     1.46
L-LC     1.06
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.18,-1.21,33.73
IT,11.71,-18.80,78.13
FINANCE,8.15,-19.09,63.15
BANKS,7.51,-13.28,61.38
MISC,5.77,-18.61,86.60
PAINTS,5.59,-16.45,39.39
HEALTHCARE,5.26,6.79,16.75
ELECTRICAL,4.71,-11.12,59.25
AUTO,4.51,-11.64,60.22


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2818335.0,23
AR,1097660.0,10
XR,1095390.0,13
X40,790349.0,10
X40N,524926.0,12
XY25,476694.0,8
OX40N,465832.0,10
SR,196222.0,2
X5K,138693.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2769822.0,28
M-SC,1471025.0,17
H-MC,1232846.0,18
H-LC,1175809.0,18
M-LC,416950.0,5
M-MC,332160.0,2
L-SC,264906.0,3
L-MC,57455.0,1
L-LC,44311.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      931156.0      6
M-SC       XY24      859972.0      7
H-SC       AR        670031.0      5
           XR        518276.0      6
H-LC       X40       509693.0      5
H-MC       XY24      485659.0      5
M-MC       XY24      332160.0      2
H-LC       AR        233798.0      2
           X40N      211184.0      5
H-SC       OX40N     204168.0      4
H-MC       XY25      203571.0      2
           X40       199504.0      4
H-SC       X40N      196966.0      4
           SR        196222.0      2
L-SC       XR        175243.0      2
M-LC       XY24      169552.0      2
M-SC       AR        161057.0      2
M-LC       X5K       131374.0      1
M-SC       OX40N     123914.0      4
           XY25      123171.0      1
           XR        121759.0      2
H-MC       X40N      116776.0      3
           XR        115387.0      1
M-LC       XR        107270.0      1
H-LC       XY25       96887.0      3
L-SC       OX40N      89663.0      1
M-SC       X40        81152.0      1
H-LC       X200       77092.0      1
L-MC       XR         57455.0      1
H-SC       MH         53003.0      1
H-MC       OX40N      48087.0      1
L-LC       XY25       44311.0      1
H-LC       XY24       39836.0      1
H-MC       AR         32774.0      1
           MH         31088.0      1
M-LC       XY25        8754.0      1
H-LC       X5K         7319.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
